In [19]:
from __future__ import print_function

import re
import random

import speech_recognition as sr 


reflections = {
    "i am": "you are",
    "i was": "you were",
    "i": "you",
    "i'm": "you are",
    "i'd": "you would",
    "i've": "you have",
    "i'll": "you will",
    "my": "your",
    "you are": "I am",
    "you were": "I was",
    "you've": "I have",
    "you'll": "I will",
    "your": "my",
    "yours": "mine",
    "you": "me",
    "me": "you",
}

pairs = [
    [
        r"my name is (.*)",
        ["Hello %1, How are you today ?",]
    ],
    [
        r"good night",
        ["Bye Bye",]
    ],
    [
        r"good (evening|morning|afternoon)",
        ["Hello","Hey there",]
    ],
     [
        r"what(.*)name ?",
        ["My name is Butler.",]
    ],
    [
        r"how are you(.*)",
        ["I'm doing good\nHow about You ?","I am good",]
    ],
    [
        r"sorry(.*)",
        ["Its alright","Its OK, never mind",]
    ],
    [
        r"go and do your work",
        ["Sure","See you later",]
    ],
    [
        r"i am good",
        ["Nice to hear that","Alright ",]
    ],
    [
        r"i am doing(.*)good", 
        ["Nice to hear that","Alright ",]
    ],
    [
        r"(.*)battery( percentage)?( percent)?( level)?",
        ["ok",]
    ],
    [
        r"(hi|hey|hello)(.*)",
        ["Hello", "Hey there",]
    ],
    [
        r"(.*)age",
        ["I was created on 27th February 2020",]
        
    ],
    [
        r"(.*)created",
        ["Siddhant created me using Python","top secret ;)",]
    ],
    [
        r"what do you do", 
        ["I am a cleaner. I can work automatically and clean entire area.",]
    ],
    [
        r"what(.*)work",
        ["I am a cleaner. I can work automatically and clean entire area.",]
    ],
    [
        r"bye-bye",
        ["BBye take care. See you soon :) ","It was nice talking to you. See you soon :)"]
    ],
    [
        r"That(.*)nice",
        ["Well, Thank you",]
    ],
    [
        r"what(.*)up",
        ["nothing much, what about you","Doing my work"]
    ],
    
    [
        r"(.*)is(.*)weather in(.*)?",
        ["Never even heard about %3",]
    ],
    [
        r"(.*)is(.*)weather(.*)?",
        ["Never even heard about %3",]
    ],

    [
        r"how(.*)health(.*)",
        ["I'm a computer program, so I'm always healthy ",]
    ],
    [
        r"quit|(good)?bye",
        ["BBye take care. See you soon :) ","It was nice talking to you. See you soon :)"]
],
]

class Chat(object):
    def __init__(self, pairs, reflections={}):
        """
        Initialize the chatbot.  Pairs is a list of patterns and responses.  Each
        pattern is a regular expression matching the user's statement or question,
        e.g. r'I like (.*)'.  For each such pattern a list of possible responses
        is given, e.g. ['Why do you like %1', 'Did you ever dislike %1'].  Material
        which is matched by parenthesized sections of the patterns (e.g. .*) is mapped to
        the numbered positions in the responses, e.g. %1.

        :type pairs: list of tuple
        :param pairs: The patterns and responses
        :type reflections: dict
        :param reflections: A mapping between first and second person expressions
        :rtype: None
        """

        self._pairs = [(re.compile(x, re.IGNORECASE), y) for (x, y) in pairs]
        self._reflections = reflections
        self._regex = self._compile_reflections()
        

    def _compile_reflections(self):
        sorted_refl = sorted(self._reflections.keys(), key=len, reverse=True)
        return re.compile(
            r"\b({0})\b".format("|".join(map(re.escape, sorted_refl))), re.IGNORECASE
        )

    def _substitute(self, str):
        """
        Substitute words in the string, according to the specified reflections,
        e.g. "I'm" -> "you are"

        :type str: str
        :param str: The string to be mapped
        :rtype: str
        """

        return self._regex.sub(
            lambda mo: self._reflections[mo.string[mo.start() : mo.end()]], str.lower()
        )

    def _wildcards(self, response, match):
        pos = response.find('%')
        while pos >= 0:
            num = int(response[pos + 1 : pos + 2])
            response = (
                response[:pos]
                + self._substitute(match.group(num))
                + response[pos + 2 :]
            )
            pos = response.find('%')
        return response

    def respond(self, str):
        """
        Generate a response to the user input.

        :type str: str
        :param str: The string to be mapped
        :rtype: str
        """

        # check each pattern
        for (pattern, response) in self._pairs:
            match = pattern.search(str)
            
            # did the pattern match?
            if match:
                resp = random.choice(response)  # pick a random response
                resp = self._wildcards(resp, match)  # process wildcards

                # fix munged punctuation at the end
                if resp[-2:] == '?.':
                    resp = resp[:-2] + '.'
                if resp[-2:] == '??':
                    resp = resp[:-2] + '?'
                return resp


    # Hold a conversation with a chatbot
    def converse(self,user_input):
        while user_input[-1] in "!.":
            user_input = user_input[:-1]
        return self.respond(user_input)
                

In [20]:
mic_name = "MacBook Air Microphone"
sample_rate = 48000
chunk_size = 2048
    
r = sr.Recognizer() 
r.pause_threshold = 0.8  # seconds of non-speaking audio before a phrase is considered complete
r.phrase_threshold = 0.3  # minimum seconds of speaking audio before we consider the speaking audio a phrase - values below this are ignored (for filtering out clicks and pops)
r.non_speaking_duration = 0.5  # seconds of non-speaking audio to keep on both sides of the recording

mic_list = sr.Microphone.list_microphone_names() 
for i, microphone_name in enumerate(mic_list): 
    if microphone_name == mic_name: 
        device_id = i
        
chat = Chat(pairs, reflections)

import subprocess
def say(text):
    subprocess.call(['say', text])


def listen_activated():
    print("say")
    with sr.Microphone(device_index = device_id, sample_rate = sample_rate, chunk_size = chunk_size) as source: 
        r.adjust_for_ambient_noise(source) 
        #print("Say Something or quit to deactivate me") 
        audio = r.listen(source) 
    
        try: 
            print("dont speak")
            text = r.recognize_google(audio)
            print(text)
            response = chat.converse(text.lower().strip())
            if response == None:
                response = "Sorry I did not understand that"
            say(response)

        except sr.UnknownValueError: 
            pass

        except sr.RequestError as e: 
            say("Hi, I am facing trouble. Please check for internet connection") 


In [25]:
while True:
    flag=listen_activated()

say
dont speak
hello
say
dont speak
how are you
say
dont speak
I am doing really good
say
dont speak
how is the weather today in Manipal
say
dont speak
bye-bye
say
dont speak
say


KeyboardInterrupt: 